In [1]:

import os
from dotenv import load_dotenv
load_dotenv()

db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')
db_host = os.environ.get('DB_HOST')
db_name = os.environ.get('DB_NAME')

os.environ['GOOGLE_API_KEY'] = os.environ.get('GOOGLE_API_KEY')

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain


In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

## SQL connection

In [6]:
# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

mysql
['adminapp_companyprofile', 'adminapp_jobapplication', 'adminapp_jobposting', 'auth_group', 'auth_group_permissions', 'auth_permission', 'auth_user', 'auth_user_groups', 'auth_user_user_permissions', 'django_admin_log', 'django_content_type', 'django_migrations', 'django_session', 'userapp_candidatedetails']

CREATE TABLE adminapp_companyprofile (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	company_name VARCHAR(255) NOT NULL, 
	company_details LONGTEXT NOT NULL, 
	logo VARCHAR(100) NOT NULL, 
	company_website VARCHAR(200) NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_companyprofile_user_id_c07b6444_fk_auth_user_id FOREIGN KEY(user_id) REFERENCES auth_user (id)
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from adminapp_companyprofile table:
id	company_name	company_details	logo	company_website	user_id
1	Botit	Robotics Company	company_logos/tf_logo_social.png	https://www.tensorflow.org/	1
2	Google	Google LLC is an American 

In [8]:
chain = create_sql_query_chain(llm, db)
chain.get_prompts()[0].pretty_print()



You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

## Table definitions and example rows

In [9]:
context = db.get_context()
print(list(context))
print(context["table_info"])

['table_info', 'table_names']

CREATE TABLE adminapp_companyprofile (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	company_name VARCHAR(255) NOT NULL, 
	company_details LONGTEXT NOT NULL, 
	logo VARCHAR(100) NOT NULL, 
	company_website VARCHAR(200) NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_companyprofile_user_id_c07b6444_fk_auth_user_id FOREIGN KEY(user_id) REFERENCES auth_user (id)
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from adminapp_companyprofile table:
id	company_name	company_details	logo	company_website	user_id
1	Botit	Robotics Company	company_logos/tf_logo_social.png	https://www.tensorflow.org/	1
2	Google	Google LLC is an American multinational corporation and technology company focusing on online advert	company_logos/iphone.jpg	https://www.google.com/	20
3	qualcomm	Qualcomm Incorporated is an American multinational corporation headquartered in San Diego, Californi	company_logos/qualcomm.png	https://www.qualcomm

In [12]:
prompt_with_context = chain.get_prompts()[0].partial(table_info=context["table_info"])
print(prompt_with_context.pretty_repr()[:1500])




You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [13]:
response = chain.invoke({"question": "How many jobs are in there"})
response

'SELECT COUNT(*) AS `num_jobs` \nFROM adminapp_jobposting;'

In [14]:
db.run(response)



'[(17,)]'

In [15]:
response = chain.invoke({"question": "Shortlist top candidates with job_id=17"})
response

'```sql\nSELECT\n  `candidate_name`,\n  `score`\nFROM `adminapp_jobapplication`\nWHERE\n  `job_id` = 17\nORDER BY\n  `score` DESC\nLIMIT 5;\n```'

In [16]:
db.run(response)


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT\n  `candidate_name`,\n  `score`\nFROM `adminapp_jobapplication`\nWHERE' at line 1")
[SQL: ```sql
SELECT
  `candidate_name`,
  `score`
FROM `adminapp_jobapplication`
WHERE
  `job_id` = 17
ORDER BY
  `score` DESC
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

## Few Short Examples

In [ ]:
examples = [
    {"input": "from job_id=19. List all applicants.", "query": "SELECT * FROM adminapp_jobapplication WHERE job_id = 19;"},
    {
        "input": "from job_id=19. Shortlist top applicants ",
        "query": "SELECT * FROM adminapp_jobapplication WHERE job_id = 19 ORDER BY score DESC LIMIT 3;",
    },
    {
        "input": "from job_id=23. Shortlist top 5 applicants",
        "query": "SELECT * FROM adminapp_jobapplication WHERE job_id = 23 ORDER BY score DESC LIMIT 5;",
    },
    {
        "input": "from job_id=14. Shortlist 5 applicants",
        "query": "SELECT * FROM adminapp_jobapplication WHERE job_id = 14 ORDER BY score DESC LIMIT 5;",
    },
    {
        "input": "from job_id=18. Shortlist applicants with score more than 80",
        "query": "SELECT * FROM adminapp_jobapplication WHERE job_id = 18 AND score > 80;",
    },
    {
        "input": "from job_id=18. Shortlist applicants with skills Lora and Fine-Tuning",
        "query": "SELECT * FROM adminapp_jobapplication WHERE (LOWER(skills) LIKE '%lora%' OR LOWER(skills) LIKE '%fine%tuning%') AND job_id = 18;",
    },
    {
        "input": "from job_id=18. Shortlist applicants with skills Lora and Fine-Tuning",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]